# `Промышленное машинное обучение на Spark`
## `Занятие 08: Spark Structured Streaming`

О чём можно узнать из этого ноутбука:

* Обработка данных в онлайн-режиме
* Подключение источников
* Spark Streaming: Filters, Join, Window

Произведём все необходимые импорты и создадим Spark-context для дальнейшей работы

In [1]:
import pyspark.sql.functions as F

from pyspark import SparkConf, SparkContext
from pyspark.sql.types import StructType, StructField
from pyspark.sql.types import StringType, DoubleType, IntegerType, LongType
from pyspark.sql import SparkSession

conf = (
    SparkConf()
        .set('spark.ui.port', '4050')
        .set('spark.driver.memory', '6g')
        .set('spark.executor.extraJavaOptions', '-Xss512m')
        .set('spark.driver.extraJavaOptions', '-Xss512m')
        .setMaster('local[*]')
        .setAppName("StructuredNetworkWordCount")
)
sc = SparkContext(conf=conf)
spark = SparkSession(sc)

Далее создадим специальный датафрейм, в который будут приходить данные по сети. Этот датафрейм представляет собой абстракцию таблицы с бесконечным числом строк, в котором все пришедшие данные располагаются в колонке `value`.

Вызов метода `readStream` указывает на то, что созданный датафрейм будет потокового типа, также при создании необходимо указать откуда будут идти данные. Источниками данных могут быть:
* Открытыте сетевые соединения
* Дириктории в локальной файловой системе или hdfs
* Из брокеров сообщений, например Kafka

In [2]:
words_lines = (spark
        .readStream
        .format('socket') # аргумент socket указывает на то, что данные будут приходить по сети
        .option('host', 'localhost') # указываем на каком хосте располагаются данные
        .option('port', '10000') # указываем на каком порте нужно вычитывать их
        .load()
)

Сделаем вывод схемы данных и типа созданного датафрейма

In [3]:
words_lines.printSchema()

f"Streaming DataFame: {words_lines.isStreaming}"

root
 |-- value: string (nullable = true)



'Streaming DataFame: True'

Далее уже можно приступать непосредственно к описании требуемой логики обработки данных. В ячейке ниже происходит операция подсчёта пришедших слов, для этого разбиваем строки на списки слов и представляем каждое слово в виде отдельной строки в абстрактной таблице.

In [4]:
words = words_lines.select(
        F.explode( # преобразуем пришедшие данные в список и каждый элемент списка преобразуем в строку датафрейма
            F.split(words_lines.value, ' ') # атрибут .value - данные которые приходят на host:port
        ).alias('word')
    )

wordCounts = words.groupBy('word').count()

В предыдущих ячейках были сформировны два из трёх шагов: вычитка и процессеинг. Теперь сформируем последний шаг - запись. В нём указываем куда пишем данные - параметр метода format, и по какой логике формируется вывод - метод outputMode.

In [5]:
query = (wordCounts
        .writeStream
        .outputMode('complete')
        .format('console')
        .start()
)

Подобно тому как исполнение операций в batch-режиме производлось после вызова action операций, в SparkStreaming вычитка новых данных происходит после запуска метода awaitTermination, в котором производится старт фоновой задачи на обновление.

In [ ]:
query.awaitTermination()

## Чтение данных из файловой системы

Попробуем считать данные из файловой системы, для этого рекомендуется заранее описать схему данных, а затем указать, какую директорию в файловой системе необходимо мониторить

In [ ]:
# описываем схему данных таблиц из директории data
userSchema = StructType(
    [
        StructField("Date", StringType(), True),
        StructField("Open", DoubleType(), True),
        StructField("High", DoubleType(), True),
        StructField("Low", DoubleType(), True),
        StructField("Close", DoubleType(), True),
        StructField("Adjusted Close", DoubleType(), True),
        StructField("Volume", DoubleType(), True),
    ]
)
# Так как информация о названии компании содержится
# в названии файла, то добавим дополнительное поле в
# датафрейм из названия файла
def get_company():
    filename = F.element_at(
        F.split(F.input_file_name(), "/"), -1
    )
    return F.element_at(F.split(filename, "_"), 1)

initDF = (spark
  .readStream
  .format("csv")
  .option("maxFilesPerTrigger", 2) # максимальное кол-во одновременно считываемых файлов
  .option("header", True)
  .option("path", "sourceDir") # путь к директории, которую необходимо отслеживать
  .schema(userSchema)
  .load()
  .withColumn("Name", get_company()) # добавим к колонкам ещё название компании
)

Задаём логику обработки данных. Производим агргацию данных об акциях компания, где будет указана самая высокая цена акции в некотором году.

In [ ]:
stock_df = (
    initDF
    .groupBy(F.col("Name"), F.year(F.col("Date")).alias("Year"))
    .agg(F.max("High").alias("Max"))
)

Задаём логику записи данных в консоль.

In [ ]:
query = (
  stock_df
  .writeStream
  .outputMode("update") # по какой логику делаем вывод - только обновления
  .option("truncate", False)
  .format("console")
  .start()
)

23/12/02 00:17:17 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-4b7d1092-3e3d-4470-9388-15f272a746ec. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
23/12/02 00:17:17 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


In [ ]:
query.awaitTermination()

Можно записывать обработанные данные также файл и в файл, но тут есть ограничения на производимые операции. Так как запись в файл поддерживает только тип записи `append` - данные внутри файлов можно добавлять, но не изменять.

In [ ]:
stock_df = (
    initDF
    .where(F.col("Close") - F.col("Open") > 0)
)

23/12/02 06:56:35 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
23/12/02 06:56:35 WARN StreamingQueryManager: Stopping existing streaming query [id=35be1d0b-031e-451b-bd48-0d58b19476c4, runId=382e6bed-abf5-4cf3-9dc1-0ae6b7216774], as a new run is being started.


In [ ]:
stock_df\
  .writeStream\
  .outputMode("append")\
  .format("csv")\
  .option("path", "output")\
  .option("header", True)\
  .option("checkpointLocation", "checkpoints/")\
  .start()\
  .awaitTermination()


### Join & Windows

Потоковые данные можно джойнить между собой или же между привычными  batch-датафреймами.

In [ ]:
companyFullNames = spark.createDataFrame(
    [
        ("AAPL", "Apple"),
        ("AMZN", "Amazon"),
        ("GOOGL","Google"),
        ("MSFT", "Microsoft"),
        ("CSCO", "CISCO")
    ], ["Name", "FullName"]
)

companyFullNames.show(2)

+----+--------+
|Name|FullName|
+----+--------+
|AAPL|   Apple|
|AMZN|  Amazon|
+----+--------+
only showing top 2 rows



In [ ]:
stock_df = (
    initDF
    .groupBy("Name", F.year("Date").alias("Year"))
    .agg(F.max("High").alias("Max"))
)

stock_df = stock_df.join(companyFullNames, on="Name", how="inner")

In [ ]:
stock_df\
.writeStream\
.outputMode("complete")\
.trigger(processingTime='1 minute') \
.option("truncate", False)\
.option("numRows", 10)\
.format("console")\
.start()\
.awaitTermination()

23/12/02 08:24:12 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-5d4b566b-e312-429a-a4e3-2002f1f38d20. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
23/12/02 08:24:12 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+----+----+------+--------+
|Name|Year|Max   |FullName|
+----+----+------+--------+
|AAPL|2007|28.99 |Apple   |
|AAPL|2010|46.67 |Apple   |
|AAPL|2013|82.16 |Apple   |
|AAPL|2015|134.54|Apple   |
|AAPL|2016|118.69|Apple   |
|AAPL|2011|60.96 |Apple   |
|AAPL|2009|30.56 |Apple   |
|AAPL|2012|100.72|Apple   |
|AAPL|2008|28.61 |Apple   |
|AAPL|2006|13.31 |Apple   |
+----+----+------+--------+
only showing top 10 rows



ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/home/evgeniy/.local/lib/python3.10/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/home/evgeniy/.local/lib/python3.10/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

Теперь попробуем создать оконную функцию, в которой будем обрабатывать данные, которые находятся в одном временном окне.

In [ ]:
lines = (
    spark
    .readStream
    .format('socket')
    .option('host', 'localhost')
    .option('port', '10000')
    .option('includeTimestamp', 'true')
    .load()
)

words = lines.select(
    F.explode(F.split(lines.value, ' ')).alias('word'),
    lines.timestamp
)

windowDuration = '1 minutes'
slideDuration = '30 seconds'

windowedCounts = words.groupBy(
    F.window(words.timestamp, windowDuration, slideDuration),
    words.word
).count().orderBy('window')

# Start running the query that prints the windowed word counts to the console
query = (windowedCounts
    .writeStream
    .outputMode('complete')
    .format('console')
    .option('truncate', 'false')
    .start()
)

query.awaitTermination()

23/12/02 08:30:13 WARN TextSocketSourceProvider: The socket source should not be used for production applications! It does not support recovery.
23/12/02 08:30:13 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-8aa4f919-d12b-44a1-9993-0d6135795916. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
23/12/02 08:30:13 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+------+----+-----+
|window|word|count|
+------+----+-----+
+------+----+-----+



-------------------------------------------
Batch: 1
-------------------------------------------
+------------------------------------------+-----+-----+
|window                                    |word |count|
+------------------------------------------+-----+-----+
|{2023-12-02 08:29:30, 2023-12-02 08:30:30}|world|1    |
|{2023-12-02 08:29:30, 2023-12-02 08:30:30}|hello|1    |
|{2023-12-02 08:30:00, 2023-12-02 08:31:00}|hello|1    |
|{2023-12-02 08:30:00, 2023-12-02 08:31:00}|world|1    |
+------------------------------------------+-----+-----+



-------------------------------------------
Batch: 2
-------------------------------------------
+------------------------------------------+-----+-----+
|window                                    |word |count|
+------------------------------------------+-----+-----+
|{2023-12-02 08:29:30, 2023-12-02 08:30:30}|world|1    |
|{2023-12-02 08:29:30, 2023-12-02 08:30:30}|hello|1    |
|{2023-12-02 08:30:00, 2023-12-02 08:31:00}|hello|2    |
|{2023-12-02 08:30:00, 2023-12-02 08:31:00}|boys |1    |
|{2023-12-02 08:30:00, 2023-12-02 08:31:00}|world|1    |
|{2023-12-02 08:30:30, 2023-12-02 08:31:30}|hello|1    |
|{2023-12-02 08:30:30, 2023-12-02 08:31:30}|boys |1    |
+------------------------------------------+-----+-----+



-------------------------------------------
Batch: 3
-------------------------------------------
+------------------------------------------+---------+-----+
|window                                    |word     |count|
+------------------------------------------+---------+-----+
|{2023-12-02 08:29:30, 2023-12-02 08:30:30}|world    |1    |
|{2023-12-02 08:29:30, 2023-12-02 08:30:30}|hello    |1    |
|{2023-12-02 08:30:00, 2023-12-02 08:31:00}|hello    |2    |
|{2023-12-02 08:30:00, 2023-12-02 08:31:00}|boys     |1    |
|{2023-12-02 08:30:00, 2023-12-02 08:31:00}|world    |1    |
|{2023-12-02 08:30:30, 2023-12-02 08:31:30}|hello    |1    |
|{2023-12-02 08:30:30, 2023-12-02 08:31:30}|boys     |1    |
|{2023-12-02 08:31:00, 2023-12-02 08:32:00}|words    |1    |
|{2023-12-02 08:31:00, 2023-12-02 08:32:00}|something|1    |
|{2023-12-02 08:31:00, 2023-12-02 08:32:00}|         |1    |
|{2023-12-02 08:31:30, 2023-12-02 08:32:30}|         |1    |
|{2023-12-02 08:31:30, 2023-12-02 08:32:30}|somet

-------------------------------------------
Batch: 4
-------------------------------------------
+------------------------------------------+---------+-----+
|window                                    |word     |count|
+------------------------------------------+---------+-----+
|{2023-12-02 08:29:30, 2023-12-02 08:30:30}|world    |1    |
|{2023-12-02 08:29:30, 2023-12-02 08:30:30}|hello    |1    |
|{2023-12-02 08:30:00, 2023-12-02 08:31:00}|hello    |2    |
|{2023-12-02 08:30:00, 2023-12-02 08:31:00}|boys     |1    |
|{2023-12-02 08:30:00, 2023-12-02 08:31:00}|world    |1    |
|{2023-12-02 08:30:30, 2023-12-02 08:31:30}|hello    |1    |
|{2023-12-02 08:30:30, 2023-12-02 08:31:30}|boys     |1    |
|{2023-12-02 08:31:00, 2023-12-02 08:32:00}|words    |1    |
|{2023-12-02 08:31:00, 2023-12-02 08:32:00}|something|1    |
|{2023-12-02 08:31:00, 2023-12-02 08:32:00}|         |1    |
|{2023-12-02 08:31:30, 2023-12-02 08:32:30}|         |1    |
|{2023-12-02 08:31:30, 2023-12-02 08:32:30}|somet

Примеры

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import from_json, col, window
from pyspark.sql.types import StructType, StringType, TimestampType

# Создание SparkSession
spark = SparkSession.builder \
    .appName("StructuredStreamingExample") \
    .getOrCreate()

# Включение логирования только ошибок
spark.sparkContext.setLogLevel("ERROR")


In [ ]:
# Чтение данных из Kafka
raw_df = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("subscribe", "events_topic") \
    .option("startingOffsets", "earliest") \
    .load()


In [ ]:
# Схема JSON-сообщения
schema = StructType() \
    .add("event_type", StringType()) \
    .add("event_time", TimestampType())

# Преобразование Kafka-сообщения
events_df = raw_df.selectExpr("CAST(value AS STRING) as json") \
    .select(from_json(col("json"), schema).alias("data")) \
    .select("data.*")


In [ ]:
# Группировка по окну и типу события
aggregated_df = events_df \
    .withWatermark("event_time", "10 minutes") \
    .groupBy(
        window(col("event_time"), "5 minutes"),
        col("event_type")
    ).count()


In [ ]:
# Потоковая запись в консоль
query = aggregated_df.writeStream \
    .outputMode("update") \  # используем режим update, чтобы не печатать всю таблицу каждый раз
    .format("console") \
    .option("truncate", False) \
    .start()

# Ожидание завершения
query.awaitTermination()


Пример ML

In [ ]:
# Подготовим модель заранее (один раз, не в стриме)

from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml.classification import LogisticRegression
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("MLModelTraining").getOrCreate()

# Обучающий датасет
train_df = spark.createDataFrame([
    (1, "A", 0.1, 0),
    (2, "B", 0.3, 1),
    (3, "A", 0.4, 0),
    (4, "B", 0.6, 1)
], ["id", "category", "feature1", "label"])

# Индексация категориального признака
indexer = StringIndexer(inputCol="category", outputCol="category_indexed")
train_df = indexer.fit(train_df).transform(train_df)

# Сбор признаков в вектор
assembler = VectorAssembler(
    inputCols=["category_indexed", "feature1"],
    outputCol="features"
)
train_df = assembler.transform(train_df)

# Обучение модели
lr = LogisticRegression(featuresCol="features", labelCol="label")
model = lr.fit(train_df)

# Сохраняем модель
model.write().overwrite().save("/tmp/lr_model")


In [ ]:
# Стриминг и применение модели

from pyspark.ml.classification import LogisticRegressionModel
from pyspark.sql.functions import col
from pyspark.ml.feature import VectorAssembler, StringIndexerModel

# Spark session
spark = SparkSession.builder.appName("StreamingML").getOrCreate()

# Чтение стрима CSV-файлов из папки
schema = "id INT, category STRING, feature1 DOUBLE"
stream_df = spark.readStream \
    .schema(schema) \
    .option("sep", ",") \
    .csv("/tmp/stream_input")  # папка, куда будут поступать новые файлы

# Загружаем модель и трансформеры
model = LogisticRegressionModel.load("/tmp/lr_model")
indexer_model = StringIndexerModel.from_labels(["A", "B"], inputCol="category", outputCol="category_indexed")
assembler = VectorAssembler(inputCols=["category_indexed", "feature1"], outputCol="features")

# Применяем предобработку и модель
indexed_df = indexer_model.transform(stream_df)
features_df = assembler.transform(indexed_df)
predictions = model.transform(features_df)

# Выводим id, вероятность и предсказанную метку
output_df = predictions.select("id", "probability", "prediction")

# Пишем в консоль (можно заменить на Parquet, Kafka и т.д.)
query = output_df.writeStream \
    .format("console") \
    .outputMode("append") \
    .start()

query.awaitTermination()
